In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import string
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

## Load in the data and set the helper variables

In [ ]:
df = pd.read_excel (r'Data/Q_data_9.xlsx')
print(df.shape)

In [ ]:
maze_data = []
dirhome = os.path.dirname(os.path.realpath('__file__'))
directory = os.path.join(dirhome, 'Data\Maze Log')
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        print(filename)
        with open(os.path.join(directory, filename)) as json_file:
            data = json.load(json_file)
            maze_data.append(data)

In [ ]:
def col2num(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num - 1

In [ ]:
GENDER = col2num("K")
ID = col2num("J")
HAND = col2num("L")
HMD = col2num("P")
VR = col2num("M")
GAMES = col2num("N")
SSQs = [col2num("Q"),col2num("Z"),col2num("AK"),col2num("AV"),col2num("BG"),col2num("BR"),col2num("CC"),col2num("CN"),col2num("CY")]
ENDQNA = col2num("DJ")
SYMPTOMS = ["General Discomfort", "Fatigue", "Headache", "Eyestrain", "Difficulty Focusing", "Blurred Vision", "Dizziness", "Vertigo"]
oculSymptoms = ("General Discomfort", "Fatigue", "Eyestrain", "Difficulty Focusing")
disSymptoms = ("Headache", "Blurred Vision", "Dizziness", "Vertigo")
participantNum = df.shape[0]

In [ ]:
print (df.iloc[:,ENDQNA])

## Basic analyses of pre experiment questions

In [ ]:
# Count males and females
males = 0
females = 0

for g in df.iloc[:,GENDER]:
    if g == "Male":
        males += 1
    else:
        females += 1

In [ ]:
# Pie chart for gender
labels = ['Female', 'Male']
sizes = [females, males]
fig1, ax1 = plt.subplots()
ax1.set_title('What is your gender?')
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
# Count Quest 1s and 2s
q1 = 0
q2 = 0

for g in df.iloc[:,HMD]:
    if g == "Quest 1":
        q1 += 1
    else:
        q2 += 1

In [ ]:
# Pie chart for gender
labels = ['Quest 1', 'Quest 2']
sizes = [q1, q2]

fig1, ax1 = plt.subplots()

ax1.set_title('Which HMD are you using?')
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
# count how many times you have played vr before
VRcount = [0,0,0,0,0]
for g in df.iloc[:,VR]:
    if g == "Never":
        VRcount[0] += 1
    elif g == "1 - 3 times":
        VRcount[1] += 1
    elif g == "4 - 6 times":
        VRcount[2] += 1
    elif g == "7 - 9 times":
        VRcount[3] += 1
    elif g == "10 or more":
        VRcount[4] += 1

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('How many times have you experienced VR before?')
time_frame = ['Never', '1 - 3 times', '4 - 6 times', '7 - 9 times', '10 or more']
ax1.bar(time_frame,VRcount)
plt.show()

In [ ]:
# count how many times you have played game before
Gamecount = [0,0,0,0,0]
for g in df.iloc[:,GAMES]:
    if g == "Never":
        Gamecount[0] += 1
    elif g == "1 - 2 times":
        Gamecount[1] += 1
    elif g == "2 -5 times":
        Gamecount[2] += 1
    elif g == "5 - 7 times":
        Gamecount[3] += 1
    elif g == "7+ times":
        Gamecount[4] += 1

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('How often do you play video games every week?')
time_frame = ['Never', '1 - 2 times', '2 -5 times', '5 - 7 times', '7+ times']
ax1.bar(time_frame,Gamecount)
plt.show()

## VRSQ analyses

In [ ]:
def feelingToNumber(vals):
    ret = []
    for val in vals:
        if val == 'Not at all':
            ret.append(0)
        elif val == 'Slightly':
            ret.append(1)
        elif val == 'Moderately':
            ret.append(2)
        elif val == 'Very':
            ret.append(3)
        else:
            print('Wrong Value')
    return ret[0] if len(ret) == 1 else ret

In [ ]:
def oculDisSplit(vals):
    """Input: df of one VRSSQ
    
    Return: Two lists with Oculomotor and Disorientation results"""
    ocul = []
    dis = []
    for i in range(len(SYMPTOMS)):
        if SYMPTOMS[i] in oculSymptoms:
            ocul.append(vals[i])
        elif SYMPTOMS[i] in disSymptoms:
            dis.append(vals[i])
    return ocul, dis

In [ ]:
def calcOculScore(vals):
    score = 0
    for val in vals:
        score += feelingToNumber([val])
    score = (score/12)*100
    return score

In [ ]:
def calcDisScore(vals):
    score = 0
    for val in vals:
        score += feelingToNumber([val])
    score = (score/15)*100
    return score

In [ ]:
def calcTotalScore(o, d):
    return (o+d)/2

In [ ]:
def calcVRSQ(result):
    ocul, dis = oculDisSplit(result)
    oculScore = calcOculScore(ocul)
    disScore = calcDisScore(dis)
    totalScore = calcTotalScore(oculScore, disScore)
    return oculScore, disScore, totalScore

In [ ]:
test = df.iloc[1,SSQs[0]:SSQs[0]+9]
print(test)
o, d, t = calcVRSQ(test)
print("Oculomotor Score: {}\nDisorientation Score: {}\nTotal Score: {}".format(o, d, t))

In [ ]:
for p in range(participantNum):
    participant = df.iloc[p,SSQs[0]:SSQs[0]+9]
    participant_val = feelingToNumber(participant)
    o, d, t = calcVRSQ(participant)
    print("Participant {}:\nOculomotor Score: {}\nDisorientation Score: {}\nTotal Score: {}\n".format(p, o, d, t))

In [ ]:
pre_experiment_vrsq_average = []
pre_experiment_vrsq_symptoms = np.zeros((participantNum, 9))
for p in range(participantNum):
    participant = df.iloc[p,SSQs[0]:SSQs[0]+9]
    participant_val = feelingToNumber(participant)
    pre_experiment_vrsq_symptoms[p,:] = participant_val
    o, d, t = calcVRSQ(participant)
    pre_experiment_vrsq_average.append([o,d,t])
pre_experiment_vrsq_average = np.array(pre_experiment_vrsq_average)
average_o = np.mean(pre_experiment_vrsq_average[:,0])
average_d = np.mean(pre_experiment_vrsq_average[:,1])
average_t = np.mean(pre_experiment_vrsq_average[:,2])
print("Average Oculomotor Score: {}\nAverage Disorientation Score: {}\nAverage Total Score: {}\n".format(average_o, average_d, average_t))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Averages')
ax1.boxplot(pre_experiment_vrsq_average, vert=False)
plt.yticks([1, 2, 3], ['Oculomotor Score', 'Disorientation Score', 'Total Score'])
plt.xlim([-10, 110])
plt.show()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Symptoms')
ax1.boxplot(pre_experiment_vrsq_symptoms, vert=False)
plt.yticks([1, 2, 3, 4, 5, 6, 7, 8, 9], ['General Discomfort', 'Fatigue', 'Headache', 'Eyestrain', 'Difficulty Focusing', 'Blurred Vision', 'Dizziness', 'Vertigo', 'Fullness of Head'])
plt.show()

In [ ]:

ids = df.iloc[:,ID]
vrsq_order = []
dict_template = {"Square":{"Teleport":0, "Dash":0,"Blur":0,"Fog":0},
                 "Circle":{"Teleport":0, "Dash":0,"Blur":0,"Fog":0}
                 }
for i in range(9):
    vrsq_order.append(dict_template)
for q, id in enumerate(ids):
    file_id = None
    for d in range(9):
        data = maze_data[d]
        for i in data['eventRecords']:
            if i['eventIdentifier'] == "ID":
                if i['eventDescription'] == str(id):
                    file_id = d
                    break
        if file_id is not None:
            break
    data = maze_data[file_id]
    ma = ""
    count = 1
    for i in data['eventRecords']:
        if i['eventIdentifier'] == "MA":
            if i['eventDescription'] == "Rotation Maze":
                    ma = "Square"
            elif i['eventDescription'] == "Back and Forth Maze":
                    ma = "Circle"
        if i['eventIdentifier'] == "MT":
            vrsq_order[q][ma][i['eventDescription']] = count
            count += 1



In [ ]:
## maze + locomotion combo
vrsq_lm = np.zeros((8,9,3))
mazes = ["Square", "Circle"]
locomotions = ["Teleport", "Dash", "Blur", "Fog"]
lm_count = 0
for m in mazes:
    for l in locomotions:
        for p in range(participantNum):
            order = vrsq_order[p][m][l]
            participant = df.iloc[p,SSQs[order]:SSQs[order]+9]
            participant_val = feelingToNumber(participant)
            pre_experiment_vrsq_symptoms[p,:] = participant_val
            o, d, t = calcVRSQ(participant)
            vrsq_lm[lm_count,p,0] = o
            vrsq_lm[lm_count,p,1] = d
            vrsq_lm[lm_count,p,2] = t
        lm_count += 1


## just locomotion
vrsq_l = np.zeros((4,18,3))
l_count = 0

for l in locomotions:
    m_count = 1
    for m in mazes:
        for p in range(participantNum):
            order = vrsq_order[p][m][l]
            participant = df.iloc[p,SSQs[order]:SSQs[order]+9]
            participant_val = feelingToNumber(participant)
            pre_experiment_vrsq_symptoms[p,:] = participant_val
            o, d, t = calcVRSQ(participant)
            vrsq_l[l_count,p*m_count,0] = o
            vrsq_l[l_count,p*m_count,1] = d
            vrsq_l[l_count,p*m_count,2] = t
        m_count += 1
    l_count += 1


## plot them on a graph

In [ ]:
#['Oculomotor Score', 'Disorientation Score', 'Total Score'])
xs = [i for i in range(8)]
plt.rcParams['figure.figsize'] = [20, 5]
for p in range(9):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    coef = np.polyfit(xs,vrsq_lm[:,p,0],1)
    poly1d_fn = np.poly1d(coef) 

    ax1.set_title("Oculomotor Scores")
    ax1.plot(xs,vrsq_lm[:,p,0])
    ax1.plot(xs, poly1d_fn(xs))
    ax1.set_ylim([-10, 100])

    coef = np.polyfit(xs,vrsq_lm[:,p,1],1)
    poly1d_fn = np.poly1d(coef) 
    ax2.set_title("Disorientation Scores")
    ax2.plot(xs,vrsq_lm[:,p,1])
    ax2.plot(xs, poly1d_fn(xs))
    ax2.set_ylim([-10, 100])

    coef = np.polyfit(xs,vrsq_lm[:,p,2],1)
    poly1d_fn = np.poly1d(coef) 
    ax3.set_title('Total Scores')
    ax3.plot(xs,vrsq_lm[:,p,2])
    ax3.plot(xs, poly1d_fn(xs))
    ax3.set_ylim([-10, 100])
    


In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
fig1, ax1 = plt.subplots()
ax1.set_title('Teleport Score')
ax1.boxplot(vrsq_l[0,:,:], vert=False)
plt.yticks([1, 2, 3], ['Oculomotor Score', 'Disorientation Score', 'Total Score'])
plt.xlim([-10, 110])
plt.show()

fig1, ax1 = plt.subplots()
ax1.set_title('Dash Scores')
ax1.boxplot(vrsq_l[1,:,:], vert=False)
plt.yticks([1, 2, 3], ['Oculomotor Score', 'Disorientation Score', 'Total Score'])
plt.xlim([-10, 110])
plt.show()

fig1, ax1 = plt.subplots()
ax1.set_title('Blur Scores')
ax1.boxplot(vrsq_l[2,:,:], vert=False)
plt.yticks([1, 2, 3], ['Oculomotor Score', 'Disorientation Score', 'Total Score'])
plt.xlim([-10, 110])
plt.show()

fig1, ax1 = plt.subplots()
ax1.set_title('Fog Scores')
ax1.boxplot(vrsq_l[3,:,:], vert=False)
plt.yticks([1, 2, 3], ['Oculomotor Score', 'Disorientation Score', 'Total Score'])
plt.xlim([-10, 110])
plt.show()

In [ ]:
stat, p = friedmanchisquare(vrsq_l[0,:,2], vrsq_l[1,:,2], vrsq_l[2,:,2])
print("Statistic: {}".format(stat))
print("P value: {}".format(p))

## Maze log analyses

In [ ]:
# Get the data from maze log

data = maze_data[0]
'''
ID - id
SR - Snap Rotation
MA - Maze type
MT - Movement type
PS - Angle

To find angle:
    first find the ps event
    Followed by MT to find the movement type

'''
# Find the angle locomotion method pair
def getAngleLocomotionPair(data):
    angleFound = False
    tempAngle = 0

    pairs = {}
    for i in data['eventRecords']:
        if i['eventIdentifier'] == "PS":
            angleFound = True
            tempAngle = float(i['eventDescription'])
        if i['eventIdentifier'] == "MT" and angleFound:
            angleFound = False
            pairs[i['eventDescription']] = tempAngle
    return pairs


def sortAngles(pairs):
    angles = []
    angles.append(pairs['Teleport'])
    angles.append(pairs['Dash'])
    angles.append(pairs['Blur'])
    angles.append(pairs['Fog'])
    return angles

def getAnlges(datas):
    angles = np.zeros((9,5))

    for i, data in enumerate(datas):
        for j in data['eventRecords']:
            if j['eventIdentifier'] == "ID":
                tempID = int(j['eventDescription'])
                break
        tempAngles = []
        pairs = getAngleLocomotionPair(data)
        tempAngles = sortAngles(pairs)
        tempAngles.append(tempID)
        angles[i,:] = tempAngles
    return angles


# Count the number of times each user used snap rotation
def userSanpRotation(data):
    count = 0
    for i in data['eventRecords']:
        if i['eventIdentifier'] == "SR":
            count +=1
    return count
# count the number of times each user used sanp rotation in different types of mazes
def mazeSnapRotation(data):
    squareMaze = 0
    circleMaze = 0
    tempCount = 0
    for i in data['eventRecords']:
        if i['eventIdentifier'] == "MA":
            if i['eventDescription'] == "Rotation Maze":
                circleMaze = tempCount
            elif i['eventDescription'] == "Back and Forth Maze":
                squareMaze = tempCount
            tempCount = 0
            tempAngle = i['eventDescription']
        if i['eventIdentifier'] == "SR":
            tempCount +=1
    return squareMaze, circleMaze

# count the number of times each user used snap rotation with each locomotion method
def locomotionSnapRotation(data):
    locomotionCount = [0,0,0,0]
    tempCount = 0
    for i in data['eventRecords']:
        if i['eventIdentifier'] == "MT":
            if i['eventDescription'] == "Teleport":
                locomotionCount[0] = tempCount
            elif i['eventDescription'] == "Dash":
                locomotionCount[1] = tempCount
            elif i['eventDescription'] == "Blur":
                locomotionCount[2] = tempCount
            elif i['eventDescription'] == "Fog":
                locomotionCount[3] = tempCount
            tempCount = 0
            tempAngle = i['eventDescription']
        if i['eventIdentifier'] == "SR":
            tempCount +=1
    return locomotionCount

In [ ]:

def calcCoordinates(angle, id, type):
    index = 0
    for i in range(9):
        if df.iloc[i,ID] == id:
            index = i
            break
    locomotions = ["Teleport", "Dash", "Blur", "Fog"]
    offset = vrsq_order[index]['Circle'][locomotions[type]]

    score = df.iloc[index,SSQs[offset] + 10]

    if score == "Strongly Disagree":
        score = 1
    elif score == "Disagree":
        score = 2
    elif score == "Somewhat Disagree":
        score = 3
    elif score == "Neutral":
        score = 4
    elif score == "Somewhat Agree":
        score = 5
    elif score == "Agree":
        score = 6
    elif score == "Strongly Agree":
        score = 7

    score = 1/7 * score

    if angle < 0:
        angle = 360 + angle
    x = score *np.sin(angle)
    y = score *np.cos(angle)
    return x, y



angles = getAnlges(maze_data)
# Plot the circle
theta = np.linspace(0, 2*np.pi, 100)

radius = 1

a = radius*np.cos(theta)
b = radius*np.sin(theta)

figure, axes = plt.subplots(1)

axes.plot(a, b)
axes.plot([0,0],[0,1], color='red') # 0 degrees
for i in range(9):
    x,y = calcCoordinates(angles[i,0], angles[i,4],0)
    axes.plot([0,x],[0,y], color='black')
axes.set_aspect(1)

plt.title('Angle difference for Teleportation')
plt.axis('off')
plt.show()

figure, axes = plt.subplots(1)

axes.plot(a, b)
axes.plot([0,0],[0,1], color='red') # 0 degrees
for i in range(9):
    x,y = calcCoordinates(angles[i,1], angles[i,4],1)
    axes.plot([0,x],[0,y], color='black')
axes.set_aspect(1)

plt.title('Angle difference for Dash')
plt.axis('off')
plt.show()

figure, axes = plt.subplots(1)

axes.plot(a, b)
axes.plot([0,0],[0,1], color='red') # 0 degrees
for i in range(9):
    x,y = calcCoordinates(angles[i,2], angles[i,4],2)
    axes.plot([0,x],[0,y], color='black')
axes.set_aspect(1)

plt.title('Angle difference for Blur')
plt.axis('off')
plt.show()

figure, axes = plt.subplots(1)

axes.plot(a, b)
axes.plot([0,0],[0,1], color='red') # 0 degrees
for i in range(9):
    x,y = calcCoordinates(angles[i,3], angles[i,4],3)
    axes.plot([0,x],[0,y], color='black')
axes.set_aspect(1)

plt.title('Angle difference for Fog')
plt.axis('off')
plt.show()

In [ ]:
srcount = [0 for _ in range(9)]
for i in range(9):
    srcount[i] = userSanpRotation(maze_data[i])
fig1, ax1 = plt.subplots()
ax1.set_title('How many times each user used snap rotation')
time_frame = ['1', '2', '3', '4', '5','6','7','8','9']
ax1.bar(time_frame,srcount)
plt.show()

In [ ]:
ss = 0
cs = 0
for i in range(9):
    s, c = mazeSnapRotation(maze_data[i])
    ss += s
    cs += c

fig1, ax1 = plt.subplots()
ax1.set_title('How many times snap rotation was used in each maze type')
time_frame = ['Square maze', 'Circle maze']
ax1.bar(time_frame,[ss,cs])
plt.show()

In [ ]:
locomotionsr = [0,0,0,0]

for i in range(9):
    l = locomotionSnapRotation(maze_data[i])
    for j in range(4):
        locomotionsr[j] += l[j]

fig1, ax1 = plt.subplots()
ax1.set_title('How many times snap rotation was used in each locomotion method')
time_frame = ['Teleport', 'Dash','Blur','Fog']
ax1.bar(time_frame,locomotionsr)
plt.show()

## Post experiment questions analyses

In [ ]:
temp = []
for i in range(4):
    test = df.iloc[:,ENDQNA+4*i:ENDQNA+4*i+4]
    temp.append(test)
temp = np.array(temp)
temp = temp.reshape(36,4)
for i in range(36):
    for j in range(4):
        if temp[i,j] == "1st":
            temp[i,j] = 1
        elif temp[i,j] == "2nd":
            temp[i,j] = 2
        elif temp[i,j] == "3rd":
            temp[i,j] = 3
        elif temp[i,j] == "4th":
            temp[i,j] = 4
stat, p = friedmanchisquare(temp[:,0], temp[:,1], temp[:,2], temp[:,3])
print("Statistic: {}".format(stat))
print("P value: {}".format(p))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('End Q&A')
ax1.boxplot(temp, vert=False)
plt.yticks([1, 2, 3,4], ['Teleport', 'Dash', 'Blur', 'Fog'])
plt.show()

In [ ]:
colum_names = {0: "Teleport", 1: "Dash",2:"Blur", 3:"Fog"}
r = sp.posthoc_nemenyi_friedman(np.array(temp, dtype='i'))
r.rename(index=colum_names ,columns=colum_names)

In [ ]:
r = sp.posthoc_conover_friedman(np.array(temp, dtype='i'))
r.rename(index=colum_names ,columns=colum_names)

In [ ]:
r = sp.posthoc_siegel_friedman(np.array(temp, dtype='i'))
r.rename(index=colum_names ,columns=colum_names)

In [ ]:
r = sp.posthoc_miller_friedman(np.array(temp, dtype='i'))
r.rename(index=colum_names ,columns=colum_names)

In [ ]:
# TODO plot the maze maps
# TODO Cross reference the resutls from mazes and responces to the questions
mazes = ["Square", "Circle"]
mazes_translation = {"Rotation Maze":"Circle", "Back and Forth Maze": "Square"}
red_zones = {"Teleport":[{2,3},{6,12,18,24,30,31,25},{35,29}], "Dash":[{1,7,8,2,3,4,5,11},{32,31,25,24,18,30},{14},{21,22},{35}], "Blur":[{1},{7,8,2,3,4,5,11,17},{10,9},{14},{32,31,25,26,24,18,30}], "Fog":[{5,11},{33,32,26,31,30},{1,2,3,4,7,8,9,10,13,14,15}]}
locomotions = ["Teleport", "Dash", "Blur", "Fog"]
cellsCir = {}
errorSquare = np.zeros((9,4))
count = 0
for i in range(17):
    for j in range(15):
        cellsCir[count] = (j,i)
cellsSQ = {}
count = 0
for i in range(6):
    for j in range(6):
        cellsSQ[count] = (j,i)
        count += 1
for q, id in enumerate(ids):
    file_id = None
    for d in range(9):
        data = maze_data[d]
        for i in data['eventRecords']:
            if i['eventIdentifier'] == "ID":
                if i['eventDescription'] == str(id):
                    file_id = d
                    break
        if file_id is not None:
            break
    data = maze_data[file_id]
    for m in mazes:
        for ll, l in enumerate(locomotions):
            MF = ""
            EM = ""
            found1 = False
            found2 = False
            for i in data['eventRecords']:
                
                if i['eventIdentifier'] == 'MF':
                    MF = i['eventDescription']
                if i['eventIdentifier'] == 'EM':
                    EM = i['eventDescription']
                if i['eventIdentifier'] == 'MA':
                    if mazes_translation[i['eventDescription']] != m:
                        continue
                    found1 = True
                if i['eventIdentifier'] == 'MT':
                    if i['eventDescription'] != l:
                        found1 = False
                        found2 = False
                        continue
                    found2 = True
                if found1 and found2:
                    break
            if found1 and found2:
                if m == "Square":
                    MF = json.loads(MF)
                    EM = json.loads(EM)
                    errorMF = 0
                    errorMFAll = 0
                    errors = []
                    newError = True

                    errorEM = 0
                    
                    for c in MF['gridLogData']:
                        errorFound = False
                        for i in range(len(red_zones[l])):
                            if c['cellId'] in red_zones[l][i]:
                                errorFound = True
                                if newError == True:
                                    newError = False
                                    errorMFAll += 1
                                    if i in errors:
                                        errorMF +=1
                                    else:
                                        errors.append(i)
                        if not errorFound:
                            newError = True

                    newError = True
                    for c in EM['gridLogData']:
                        errorFound = False
                        for i in range(len(red_zones[l])):
                            if c['cellId'] in red_zones[l][i]:
                                errorFound = True
                                if newError == True:
                                    newError = False
                                    errorEM +=1

                        if not errorFound:
                            newError = True
                elif m == "Cirlce":
                    EM = json.loads(EM)
            else:
                print('Something went wrong')
            if m == "Square":
                errorSquare[q,ll] = errorMF + errorEM

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Errors')
ax1.boxplot(errorSquare, vert=False)
plt.yticks([1, 2, 3,4], ['Teleport', 'Dash', 'Blur', 'Fog'])
plt.show()

In [ ]:
cells = {}
count = 0
for i in range(15):
    for j in range(17):
        cells[count] = (i,j)
        count += 1
x = []
y = []
#for c in MF['gridLogData']:
#    x.append(cells[c['cellId']][0])
#    #print(c['cellId'])
#    y.append(cells[c['cellId']][1])
for c in EM['gridLogData']:
    x.append(cells[c['cellId']][0])
    #print(c['cellId'])
    y.append(cells[c['cellId']][1])

errorMF = 0
errorMFAll = 0
errors = []
newError = True

errorEM = 0
'''
for c in MF['gridLogData']:
    errorFound = False
    for i in range(len(red_zones[l])):
        if c['cellId'] in red_zones[l][i]:
            errorFound = True
            if newError == True:
                newError = False
                errorMFAll += 1
                if i in errors:
                    errorMF +=1
                else:
                    errors.append(i)
    if not errorFound:
        newError = True
print(errorMF)
print(errorMFAll)
newError = True
for c in EM['gridLogData']:
    errorFound = False
    for i in range(len(red_zones[l])):
        if c['cellId'] in red_zones[l][i]:
            errorFound = True
            if newError == True:
                newError = False
                errorEM +=1

    if not errorFound:
        newError = True
print(errorEM)
'''

In [ ]:
plt.plot(x, y)